In [ ]:
import glob
import json
import os

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import skimage.io as io
from pycocotools.coco import COCO

## #1 Create labels

In [ ]:
json_path = '/root/data/small_pen_data_collection/coco_body_parts_misrectified_labeled.json'

In [ ]:
# # coco fix
cocofile = json.load(open(json_path))

# images = glob.glob('/root/data/small_pen_data_collection/*_rectified/*.jpg')
# image_dict = {}
# for img in images:
#     image_dict[os.path.basename(img)] = img
    
# for img in cocofile:
#     img_name = img['Labeled Data'].split('%2F')[-1].split('?')[0]
#     img['local_path'] = image_dict[img_name]
    
# with open(json_path, 'w') as f:
#     json.dump(cocofile, f)

In [ ]:
print(len(cocofile['annotations']))
print(len(cocofile['images']))

In [ ]:
coco = COCO(json_path)
image_ids = coco.getImgIds()
cats = coco.cats
random_id = image_ids[np.random.randint(0, len(image_ids))]
print(random_id)
image_data = coco.loadImgs([random_id])[0]
annotation_ids = coco.getAnnIds(imgIds=[image_data['id']])
annotations = coco.loadAnns(annotation_ids)

In [ ]:
# load and display instance annotations
image = io.imread(image_data['local_path'].replace("sotra-small-pen_0", "small-pen-test-site_1"))
f ,ax = plt.subplots(1, figsize=(20, 20))
ax.imshow(image); 


coco.showAnns(annotations)


for ann in annotations:
    bbox = ann['bbox']
    rec = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], 
                            edgecolor='r', facecolor=None, fill=False, linestyle='--', linewidth=2)
    ax.add_patch(rec)
    # cat_name = [c['name'] for c in cats if c['id'] == ann['category_id']][0]
    cat_name = cats[ann['category_id']]['name']
    ax.text(bbox[0], bbox[1]-10, cat_name, fontsize=16, color='w')
    
    
plt.axis('off')
plt.show()

In [ ]:
eye_center_data = []

In [ ]:
# create eye center annotation
image_ids = coco.getImgIds()
cats = coco.cats
eid = 1
for image_id in image_ids:
    tmp = {}
    random_id = image_ids[np.random.randint(0, len(image_ids))]
    image_data = coco.loadImgs([random_id])[0]
    if "local_path" not in image_data:
        continue
    tmp['local_path'] = image_data['local_path'].replace("sotra-small-pen_0", "small-pen-test-site_1")
    tmp['id'] = eid
    tmp['category'] = "eye center"
    if len(coco.getAnnIds(imgIds=[image_data['id']], catIds=[2])) > 0:
        
        annotation_ids = coco.getAnnIds(imgIds=[image_data['id']] ) # , catIds=[2])
        eyeid = coco.getAnnIds(imgIds=[image_data['id']], catIds=[2])
        eye = coco.loadAnns(eyeid)[0]
        annotations = coco.loadAnns(annotation_ids)
        tmp['bbox'] = eye['bbox']
        tmp["center"] = [eye['bbox'][0] + eye['bbox'][2]/2.0, eye['bbox'][1] + eye['bbox'][3]/2.0]
        
        eye_center_data.append(tmp)

In [ ]:
with open('/root/data/small_pen_data_collection/eye_center_labeled.json', 'w') as  f:
    json.dump(eye_center_data, f)

## #1 First approach - images chunks

In [ ]:
import glob
import json
import os

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import skimage.io as io
from pycocotools.coco import COCO

import cv2
import numpy as np

In [ ]:
with open('/root/data/small_pen_data_collection/eye_center_labeled.json', 'r') as f:
    eye_center = json.load(f)

In [ ]:
# focus on eye detection first

In [ ]:
random_eye = np.random.choice(eye_center)

In [ ]:
random_eye = eye_center[0]

In [ ]:
image = io.imread(random_eye['local_path'])
f, ax = plt.subplots(1, figsize=(20, 10))
ax.imshow(image)
circle = patches.Circle(random_eye['center'], radius=10, color='r')
ax.add_patch(circle)
plt.show()

# labels = np.zeros((fish_only.shape[0], fish_only.shape[1]))
# c = 1
# for (class_name, point) in image_data['Label'].items():
#     if class_name != 'Eye Center':
#         continue
#     print(class_name, c)
#     loc = point[0]['geometry']
#     labels[loc['y']-1300, loc['x']] = c
#     c += 1

In [ ]:
# ## first create dataset
# eye_tiles = []
# other_tiles = [] # let's make this 100 times bigger (and only sample around the fish)
# ssize = 64
# for ec in eye_center:
#     # load the image
#     image = io.imread(ec['local_path'])
    
#     # create the tiles
#     # tiles_coordinates = [(y, x) for x in range(0, 4096, ssize) for y in range(0, 3000, ssize)]
#     eye_center_coordinates = ec['center']
#     eye_bbox = ec['bbox']
#     x1, y1, x2, y2 = eye_bbox[1], eye_bbox[0], eye_bbox[1]+eye_bbox[3], eye_bbox[0]+eye_bbox[2]
#     eye_tiles_x = list(set([int(x1 - x1 % ssize), int(x2 - x2 % ssize)]))
#     eye_tiles_y = list(set([int(y1 - y1 % ssize), int(y2 - y2 % ssize)]))
    
#     for ex in eye_tiles_x:
#         for ey in eye_tiles_y:
            
    
# #     for ex in eye_tiles_x:
# #         # get a random square close by
# #         rex = list(range(ex - 10*ssize, ex, 64)) + list(range(ex + ssize, ex + 10*ssize, 64))
# #         random_x= np.random.choice(rex, size=2, replace=False)
# #         for ey in eye_tiles_y:
# #             eye_tiles.append([ec['local_path'], ex, ey])
# #             rey = list(range(ey - 10*ssize, ey, 64)) + list(range(ey + ssize, ey + 10*ssize, 64))
# #             random_y = np.random.choice(rey, size=2, replace=False)
# #             for x in random_x:
# #                 for y in random_y:
# #                     if x > 0 and y > 0 and x < 3000 and y < 4096:
# #                         other_tiles.append([ec['local_path'], x, y])
                    
            
# #     eye_center_tile = [eye_center_coordinates[0] - eye_center_coordinates[0] % 64, 
# #                        eye_center_coordinates[1] - eye_center_coordinates[1] % 64]


In [ ]:
# eye_tiles = []
# other_tiles = [] # let's make this 100 times bigger (and only sample around the fish)
# ssize = 64
# for ec in eye_center:  
#     # create the tiles
#     ecy, ecx = ec['center']
#     tilex = int(ecx - ecx % ssize)
#     tiley = int(ecy - ecy % ssize)
#     eye_tile = [ec['local_path'],
#                 tilex,
#                 tiley,
#                 int(ecx%64),
#                 int(ecy%64)]
#     eye_tiles.append(eye_tile)
    
#     # get a random square close by
#     rex = list(range(tilex - 10*ssize, tilex, 64)) + list(range(tilex + ssize, tilex + 10*ssize, 64))
#     random_x= np.random.choice(rex, size=10, replace=False)
    
#     rey = list(range(tiley - 10*ssize, tiley, 64)) + list(range(tiley + ssize, tiley + 10*ssize, 64))
#     random_y = np.random.choice(rey, size=10, replace=False)
    
#     for x in random_x:
#         for y in random_y:
#             if x > 0 and y > 0 and x < 3000 and y < 4096:
#                 other_tiles.append([ec['local_path'], x, y])


In [ ]:
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# plt.imshow(io.imread('/root/data/small_pen_data_collection/eyetiles128/left_small-pen-test-site_1_1538490386516.jpg_26_20.jpg'))

In [ ]:
eye_tiles = []
other_tiles = [] # let's make this 100 times bigger (and only sample around the fish)
ssize = 32
for ec in tqdm(eye_center):  
    # create the tiles
    ecy, ecx = ec['center']
    # print(ec['center'])
    tilex = int(ecx - ecx % ssize)
    tiley = int(ecy - ecy % ssize)
    img = io.imread(ec['local_path'])
    tile = img[tilex:tilex+ssize, tiley:tiley+ssize]
    if tile.shape[0] == ssize and tile.shape[1] == ssize: 
        io.imsave('/root/data/small_pen_data_collection/eyetiles32/{}_{}_{}.jpg'.format(os.path.basename(ec['local_path']),
                                                                                      int(ecx%ssize), int(ecy%ssize)), tile)
    eye_tile = [tile,
                int(ecx%ssize),
                int(ecy%ssize)]
    eye_tiles.append(eye_tile)
    
    # get a random square close by
    rex = list(range(tilex - 5*ssize, tilex, ssize)) + list(range(tilex + ssize, tilex + 5*ssize, ssize))
    random_x= np.random.choice(rex, size=2, replace=False)
    
    rey = list(range(tiley - 5*ssize, tiley, ssize)) + list(range(tiley + ssize, tiley + 5*ssize, ssize))
    random_y = np.random.choice(rey, size=2, replace=False)
    
    ct = 0
    for x in random_x:
        for y in random_y:
            if x > 0 and y > 0 and x < 3000 and y < 4096:
                tile = img[x:x+ssize, y:y+ssize]
                if tile.shape[0] == ssize and tile.shape[1] == ssize:
                    io.imsave('/root/data/small_pen_data_collection/othertiles32/{}_{}.jpg'.format(os.path.basename(ec['local_path']), ct), tile)
                other_tiles.append([tile])
                ct += 1


In [ ]:
img = io.imread('/root/data/small_pen_data_collection/181001010008/right_small-pen-test-site_1_1538488944389.jpg')
plt.imshow(img[1408:1408+64, 1152:1152+64, :])
plt.show()

In [ ]:
print(len(eye_tiles))

In [ ]:
print(len(other_tiles))

In [ ]:
eye_tiles_x

In [ ]:
eye_tiles_y

In [ ]:
plt.imshow(image[1408:1408+64, 1152:1152+64])
plt.show()

In [ ]:
plt.imshow(image[1472:1472+64, 1152:1152+64])
plt.show()

In [ ]:
plt.imshow(image[1408:1408+128, 1152:1152+128])
plt.show()

In [ ]:
def generator(eye_tiles, other_tiles, batch_size=32, nclasses=2, ssize=64):
    # image_resize = cv2.resize(image, (192, 64))

    while True:     
        x_batch = np.zeros((batch_size, ssize, ssize, 3))
        y_batch_class = np.zeros((batch_size, nclasses)) 
        y_batch_reg = np.zeros((batch_size, nclasses))
        for i in range(batch_size):
            coin = np.random.rand()
            if coin > 0.5:
                lab = eye_tiles[np.random.randint(len(eye_tiles))]
                # random_tile = io.imread(lab[0])[lab[1]:lab[1]+64, lab[2]:lab[2]+64]
                random_tile = lab[0]
#                 y_batch_reg[i, 0] = lab[3]
#                 y_batch_reg[i, 1] = lab[4]
                y_batch_reg[i, 0] = lab[1]
                y_batch_reg[i, 1] = lab[2]
            else:
                lab = other_tiles[np.random.randint(len(other_tiles))]
                # random_tile = io.imread(lab[0])[lab[1]:lab[1]+64, lab[2]:lab[2]+64]
                random_tile = lab[0]
            x_batch[i,...] = random_tile
            y_batch_class[i, int(np.round(coin))] = 1 # smart uh :)

        yield x_batch, {"class": y_batch_class, "reg": y_batch_reg}
        
    
    
    

# START HERE - CREATE GENERATOR



In [ ]:
import glob
import json
import os

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import skimage.io as io
from pycocotools.coco import COCO

import cv2
import numpy as np

In [ ]:
def generator(eye_tiles, other_tiles, batch_size=16, nclasses=2, ssize=64):
    while True:    
        x_batch = np.zeros((batch_size, ssize, ssize, 3))
        y_batch_class = np.zeros((batch_size, nclasses)) 
        y_batch_reg = np.zeros((batch_size, nclasses))
        for i in range(batch_size):
            coin = np.random.rand()
            if coin > 0.5:
                tile_path = np.random.choice(eye_tiles)
                random_tile = io.imread(tile_path)
                y_batch_reg[i, 0] = int(os.path.basename(tile_path).split('.')[-2].split('_')[1])
                y_batch_reg[i, 1] = int(os.path.basename(tile_path).split('.')[-2].split('_')[2])
            else:
                tile_path = np.random.choice(other_tiles)
                random_tile = io.imread(tile_path)
            x_batch[i,...] = np.array(random_tile, dtype= np.uint8)
            y_batch_class[i, int(np.round(coin))] = 1 # smart uh :)
        x_batch = np.array(x_batch, dtype=np.uint8)
        yield x_batch, {"class": y_batch_class, "reg": y_batch_reg}

In [ ]:
ssize = 32

In [ ]:
eye_tiles= glob.glob('/root/data/small_pen_data_collection/eyetiles32/*.jpg')
other_tiles= glob.glob('/root/data/small_pen_data_collection/othertiles32/*.jpg')
train_gen = generator(eye_tiles[:512], other_tiles[:1197], ssize=ssize)
val_gen = generator(eye_tiles[512:], other_tiles[1197:], ssize=ssize)

In [ ]:
xb, yb = next(train_gen)
for i in range(32):
    plt.imshow(xb[i, ...])
    if np.argmax(yb["class"][i, :]) == 0:
        plt.title("random")
    else:
        plt.title("eye")
    plt.show()

In [ ]:
import keras
from keras.layers import Conv2D, Flatten, Dense, Input, MaxPool2D, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam 

In [ ]:
# !nvidia-smi

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
inp = Input(shape=(ssize, ssize, 3))
x = Conv2D(filters=64, kernel_size=(3, 3))(inp)
x = Conv2D(filters=64, kernel_size=(3, 3))(x)
x = Conv2D(filters=64, kernel_size=(3, 3))(x)
x = MaxPool2D()(x)
x = BatchNormalization()(x)

x = Conv2D(filters=128, kernel_size=(3, 3))(x)
x = Conv2D(filters=128, kernel_size=(3, 3))(x)
x = Conv2D(filters=128, kernel_size=(3, 3))(x)
x = MaxPool2D()(x)
x = BatchNormalization()(x)

x = Flatten()(x)
x = Dense(128)(x)
x = BatchNormalization()(x)
x = Dense(256)(x)
x = BatchNormalization()(x)

classifer = Dense(2, activation="softmax", name="class")(x)
regressor = Dense(2, activation="linear", name="reg")(x)

model = Model(inputs=[inp], outputs=[classifer, regressor])
# print(model.summary())

In [ ]:
model.compile(optimizer="adam", loss={"class": "categorical_crossentropy", "reg": "mean_absolute_error"})

In [ ]:
history = model.fit_generator(train_gen,
                    steps_per_epoch= 512 // 16, 
                    epochs=500,
                    validation_data=val_gen,
                    validation_steps=128// 16)

# predict test

In [ ]:
eye = xb[0, ...].squeeze().astype(dtype=np.uint8)
plt.imshow(eye)
plt.show()

In [ ]:
# %matplotlib notebook 

In [ ]:
R = model.predict_on_batch(np.expand_dims(xb[0, ...], axis=0))

In [ ]:
R[1]

In [ ]:
im = fish_only
ssize = 32
tiles = [(im[x:x+ssize, y:y+ssize], labels[x:x+ssize, y:y+ssize], (x,y)) for x in range(0,im.shape[0], ssize) for y in range(0,im.shape[1], ssize)]

In [ ]:
for t in tiles:
    if np.sum(t[1]) > 0:
        print(t[2])

In [ ]:
f, ax = plt.subplots(figsize=(20, 10))
ax.imshow(fish_only[200:800, : 800])
# ax.margins(1000)
circle = patches.Circle((414, 529-200), radius=10, color='r')
ax.add_patch(circle)
circle = patches.Circle((384+R[1].squeeze()[0], 512+R[1].squeeze()[1]-200), radius=10, color='b')
ax.add_patch(circle)
plt.show()

In [ ]:
R